In [1]:
from transformers.generation_utils import *

In [ ]:
import os
import sys
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None)
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
import transformers
import datasets

print(f"Running on transformers v{transformers.__version__} and datasets v{datasets.__version__}")

In [ ]:
sys.path.append('../src-py')

import transformers
import datasets
from mt_bart_v2 import *

print(f"Running on transformers v{transformers.__version__} and datasets v{datasets.__version__}")

In [ ]:
tokenizer = BartTokenizer.from_pretrained('../data/output/joint-con-counter-bart-model-no-attention/')

#joint_attention_model = BartModelV2.from_pretrained('../data/output/joint-con-counter-bart-model-with-joint-attention/', conc_loss_weight = 0.2, counter_loss_weight=0.8, attention_to_conc=True, conc_decoder=True).to(device)
nojoint_attention_model     = BartModelV2.from_pretrained('../data/output/joint-con-counter-bart-model-no-attention-finetuned/0-1-1-0/', conc_loss_weight = 0.1, counter_loss_weight=1.0, attention_to_conc=False, conc_decoder=True).to(device)
#noconc_decoder_model = BartModelV2.from_pretrained('../data/output/joint-con-counter-bart-model-no-conc-decoder/', conc_loss_weight = 0.2, counter_loss_weight=0.8, attention_to_conc=False, conc_decoder=False).to(device)

# tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
# model     = BartModelV2.from_pretrained('facebook/bart-base', conc_loss_weight = 0.5, counter_loss_weight=0.5).to(device)
# original_bart_model = BartModel.from_pretrained('facebook/bart-base').to(device)
# #load the weights of the two decoders
# model.conclusion_decoder.load_state_dict(original_bart_model.decoder.state_dict())
# model.counter_decoder.load_state_dict(original_bart_model.decoder.state_dict())

### Evaluating on Eval data:

In [6]:
gen_kwargs = {
    "do_sample": True, 
    "max_length":200,
    "top_p":0.95, 
    "top_k":50,
    "num_beams":1
}

In [7]:
ceph_dir = '/home/sile2804/data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation'
local_home_dir = '../data'

valid_df = pd.read_pickle(ceph_dir + '/reddit_data/conclusion_and_ca_generation/valid_conclusion_comp_remove_75sem_perc_with_targets.pkl')

In [8]:
valid_sample = valid_df.sample(5)

In [9]:
premises = [' '.join(x) for x in valid_sample.masked_premises.tolist()]
premise_input_ids = [tokenizer([x], return_tensors="pt").input_ids.to(device) for x in premises]

In [10]:
#valid_sample['noconc_counters']  = [tokenizer.decode(noconc_decoder_model.generate(x, **gen_kwargs)[1][0], skip_special_tokens=True) for x in premise_input_ids]
valid_sample['con_noattention_counters']  = [tokenizer.decode(nojoint_attention_model.generate(x, **gen_kwargs)[1][0], skip_special_tokens=True) for x in premise_input_ids]
#valid_sample['full_model_counters']  = [tokenizer.decode(joint_attention_model.generate(x, **gen_kwargs)[1][0], skip_special_tokens=True) for x in premise_input_ids]

In [11]:
valid_sample[['masked_premises', 'title', 'con_noattention_counters']].head()

,masked_premises,title,con_noattention_counters
392059,"[i m a republican., but i believe that anyone can be married to anyone., as long as it doesn t hinder my rights you can marry whoever you want., you can spend the rest of your life with the any person you desire., the republican party believes in traditional marriage and voted against gay marriage in 2015., i can t believe it was not legal in the first place., things like that make me question the party continuously., my major beliefs belong to the republican party., therefore, i call myself republican., due to the political stereotypes running around, i sometimes call myself a centrist., i get slandered all the time by republicans calling me traitor or flat out ignoring me., i get assigned to the long belief list on the republican website., i actually hate many things republicans in power do., i don t like trump., i ve seen enough., this doesn t just apply to republicans though., it applies to all parties., i only know about the republican side because of my experience., we should stop assuming that people saying that they belong to a party automatically believes in everything it stands for.]",People should stop assuming that someone in a political party believes EVERYTHING that party believes in.,i'm a man and not a man. i am a woman. i should be able to keep my family around because i'm a dog.
404067,"[i want to preface this that i like the idea behind automatic voter registration., it helps encourage the political process., but specifically, in my home state of florida, under the current system this would not be a good idea, for one main reason., in florida, the voter roll is public information., anyone can request a disk containing the entire voter roll including names, addresses, birthdays, mailing addresses, gender, race, etc. for free., there are several sites that offer searchable databases of this data., even if you could get your data off these sites some sites refuse to take stuff down , you also would have to file an exemption request, the only options available to a common citizen are: victim of sexual battery, aggravated child abuse, aggravated stalking, harassment, aggravated battery, or domestic violence., official verification of the crime is required certified participant in address confidentiality program only for victims of domestic violence you cannot simply request an exemption just because you want to be anonymous, or because you are afraid of a person who hasn't committed a specific provable crime against you., with automatic voter registration, we'd be putting a large amount of personal information in a public database, most likely without the knowledge of those being registered., this is a massive breach of privacy and can be a danger to society., as this is automatic, this database will begin to look more and more like a database of the names and addresses of the majority of floridians., a white supremacist could use the database to target ethnic minorities, as race is included in the database a person could use the information in the database to possibly commit identity theft a person could be stalked using the information in the database a victim of a crime, such as rape, that they did not report or that could not be prosecuted, could be found by their abuser a person of general popularity, such as an internet celebrity, could be doxxed quite simply, florida needs the way it handles i's citizens' personal data before implementing automatic voter registration.]",Automatic voter registration is NOT a good idea (specifically in Florida),"it's not like 'it's important to vote because you're just talking about people with nothing to do with themselves.'it's not that the majority of people need to be able to be taken down. it's that the government has already been responsible for being able to afford, therefore we are not talking about all the people who need to have control of this."
408080,"[a lot of people seem to think it's rude to correct some